Import modułów i pakietów:

In [ ]:
import numpy as np
from keras import Sequential
from keras.layers import Dense
from sklearn import metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder, LabelEncoder



Wczytywanie danych z pliku .pkl:
  1. Wykorzystanie funkcji read_pickle z pakietu pandas
  2. Wyodrębnienie wektorów cech do macierzy X
  3. Wyodrębnienie etykiet kategorii do wektora y


In [ ]:
# Create empty lists for features and labels
X = []
y = []

# Read features from the .pkl file
features_from_pkl_file = pd.read_pickle('results.pkl')

# Define the properties to extract from the file
properties_from_file = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']

# Extract unique iter_texture categories
T_images = features_from_pkl_file['category'].unique()

# Iterate over each iter_texture category
for iter_texture in T_images:
    # Filter the data for the current iter_texture category
    texture_data = features_from_pkl_file[features_from_pkl_file['category'] == iter_texture]

    # Extract the properties for the current iter_texture category
    properties = texture_data[properties_from_file].values

    # Flatten and add the properties to the X list
    X.extend([np.hstack(prop).flatten() for prop in properties])

    # Add labels for each sample in the current iter_texture category
    y.extend([iter_texture] * len(properties))

Wykonanie kodowania całkowitoliczbowego dla wektora y → y_int:

In [ ]:
label_encoder = LabelEncoder() # Create an instance of LabelEncoder
y_int = label_encoder.fit_transform(y) # Encode categorical labels y into numeric values and store them in y_int

Wykonanie kodowania 1 z n dla wektora y_int → y_onehot:

In [ ]:
onehot_encoder = OneHotEncoder(sparse_output=False) # Create an instance of OneHotEncoder with dense output
y_onehot = onehot_encoder.fit_transform(y_int.reshape(-1, 1)) # One-hot encode the integer-encoded labels y_int

Podzielenie zbioru X oraz wektora etykiet y_onehot na część treningową (70%) i testową (30%):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.3) # Split the dataset and one-hot encoded labels into training and testing sets

Tworzenie modelu sieci neuronowej:

  1. Utworzenie obiektu sieci typu Sequential
  2. Dodanie dwóch warstw typu Dense. W pierwszej warstwie ustawienie liczby perceptronów na 10, funkcja aktywacji - sigmoid, wymiar wejściowy - input_dim = 72 (tyle, ile jest cech w zbiorze danych). W drugiej warstwie ustawienie liczby neuronów na 3 (tyle ile jest klas) oraz funkcji aktywacji na softmax.
  3. Skompilowanie modelu z funkcją straty categorical_crossentropy oraz algorytmem optymalizacji: sgd (=stochastic gradient descent). Dodanie accuracy jako metryki wyliczanej w trakcie uczenia.



In [ ]:
model = Sequential() # Create a sequential model
model.add(Dense(10, activation='sigmoid', input_dim=72)) # Add a dense layer with 10 units, sigmoid activation, and input dimension of 72
model.add(Dense(3, activation='softmax')) # Add a dense layer with 3 units and softmax activation
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) # Compile the model with categorical cross-entropy loss, SGD optimizer, and accuracy metric

Uczenie sieci - wywołanie na rzecz modelu funkcji fit:

In [ ]:
X_train = np.asarray(X_train).astype(np.float32) # Convert X_train to a NumPy array of type float32
model.fit(X_train, y_train, epochs=100, batch_size=10, shuffle=True) # Fit the model to the training data

Epoch 1/100
4/4 [==============================] - 0s 4ms/step - loss: 2.1880 - accuracy: 0.1562
Epoch 2/100
4/4 [==============================] - 0s 4ms/step - loss: 1.8885 - accuracy: 0.1562
Epoch 3/100
4/4 [==============================] - 0s 4ms/step - loss: 1.6899 - accuracy: 0.1562
Epoch 4/100
4/4 [==============================] - 0s 4ms/step - loss: 1.5380 - accuracy: 0.1562
Epoch 5/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3957 - accuracy: 0.1562
Epoch 6/100
4/4 [==============================] - 0s 4ms/step - loss: 1.3054 - accuracy: 0.1562
Epoch 7/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1881 - accuracy: 0.1562
Epoch 8/100
4/4 [==============================] - 0s 4ms/step - loss: 1.1051 - accuracy: 0.1562
Epoch 9/100
4/4 [==============================] - 0s 5ms/step - loss: 1.0250 - accuracy: 0.1562
Epoch 10/100
4/4 [==============================] - 0s 6ms/step - loss: 0.9612 - accuracy: 0.3438
Epoch 11/100
4/4 [===========

Testowanie sieci:

  1. Przekonwertowanie wektora y_test oraz y_pred do kodowania całkowitego.
  2. Na podstawie otrzymanych wektorów etykiet całkowitoliczbowych wyliczenie macierz pomyłek.



In [ ]:
X_test = np.asarray(X_test) # Convert X_test to a NumPy array
y_pred = model.predict(X_test) # Make predictions on the test data
y_test_int = np.argmax(y_test, axis=1) # Convert the one-hot encoded test labels to integer labels
y_pred_int = np.argmax(y_pred, axis=1) # Convert the predicted probabilities to integer labels
cm = metrics.confusion_matrix(y_test_int, y_pred_int) # Compute the confusion matrix
print(cm)

1/1 [==============================] - 0s 55ms/step
[[4 0 0]
 [0 3 0]
 [0 0 7]]
